In [1]:
import csv
import re
import pandas as pd
import time

import selenium 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options

In [2]:
sample_df = pd.read_csv("inxight_data_2.csv")
cas_df = sample_df[sample_df["CAS"] != "No CAS info"]
cas_df

,Unnamed: 0,CAS,URL,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,0,51022-69-6,https://drugs.ncats.io/drug/423W026MA9,US Approved Rx 2003,ANDA076065,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1979,CYCLOCORT by ASTELLAS,https://www.accessdata.fda.gov/scripts/cder/da...,Corticosteroid-responsive dermatoses,Approved
1,1,90-84-6,https://drugs.ncats.io/drug/Q94YYU22B8,US Approved Rx 2011,ANDA091680,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1959,TEPANIL by 3M,https://www.accessdata.fda.gov/scripts/cder/da...,Unknown; Exogenous obesity; Unknown,Approved
2,2,300-62-9,https://drugs.ncats.io/drug/CK833KGX7E,US Approved Rx 2003,ANDA040480,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1931,Benzedrine Inhaler,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,Attention deficit hyperactivity disorder,Approved
3,3,20537-88-6,https://drugs.ncats.io/drug/ILA426L95O,US Approved Rx 1995,NDA020221,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1995,NDA020221,https://www.accessdata.fda.gov/scripts/cder/da...,Ovarian cancer; Head and neck cancer,Approved; Approved
4,4,37517-28-5,https://drugs.ncats.io/drug/84319SGC3C,US Approved Rx 1993,ANDA064045,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1976,AMIKIN by APOTHECON,https://www.accessdata.fda.gov/scripts/cder/da...,Cystic fibrosis; Sepsis; Urinary tract infecti...,Phase III; Approved; Approved; Approved; Phase...
...,...,...,...,...,...,...,...,...,...,...,...
828,828,151319-34-5,https://drugs.ncats.io/drug/S62U433RMH,US Approved Rx 2008,ANDA077505,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1999,NDA020859,https://www.accessdata.fda.gov/scripts/cder/da...,Insomnia,Approved
829,829,139110-80-8,https://drugs.ncats.io/drug/L6O3XI777I,US Approved Rx 1999,NDA021036,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1999,NDA021036,https://www.accessdata.fda.gov/scripts/cder/da...,Influenza virus infections,Approved
830,830,144-68-3,https://drugs.ncats.io/drug/CV0IB81ORO,Possibly Marketed Outside US,"Ocuvel by Adler-Stern Pharmaceuticals, LLC",https://www.drugbank.ca/drugs/DB11176,Possibly Marketed Outside US 2014,"Ocuvel by Adler-Stern Pharmaceuticals, LLC",https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Vision disorders; Cataract; Age-related memory...,Approved; Phase III; Phase III
831,831,107452-89-1,https://drugs.ncats.io/drug/7I64C51O16,US Approved Rx 2004,NDA021060,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2004,NDA021060,https://www.accessdata.fda.gov/scripts/cder/da...,severe chronic pain,Approved


In [3]:
columns_to_check = [
    'URL',
    'Highest Development Event Status and Year',
    'Highest Development Event Source ID',
    'Highest Development Event Source URL',
    'Earliest Approved Event Status and Year',
    'Earliest Approved Event Source ID',
    'Earliest Approved Event Source URL',
    'conditions',
    'phases'
]

In [4]:
filtered_na = cas_df[cas_df[columns_to_check].isna().all(axis=1)]

In [5]:
cas_arr = filtered_na['CAS'].tolist()
len(cas_arr)

241

In [6]:
def setup_webdriver():
    '''Initializes a headless selenium webdriver'''
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def url_inxight(cas):
    '''Generates an Inxight URL for a given CAS'''
    return f"https://drugs.ncats.io/substances?q={cas}"

# def url_inxight(cas):
#     '''Generates an Inxight URL for a given CAS'''
#     return f"https://drugs.ncats.io/substances?q=%22{cas}%22&facet=Substance%20Form%2FPrincipal%20Form"

In [7]:
def get_inxight_url(molecule, driver):
    '''
    Parameters
    ----------
    A molecule's CAS and an initialised webdriver.

    Returns
    -------
    The top Inxight search result for a given CAS number
    '''
    
    url = "N/A"  # Default in case of failure

    driver.get(url_inxight(molecule))
    try:
        WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[id="card-title"]')))
        elements = driver.find_elements(By.CSS_SELECTOR, 'a[id="card-title"]')
        if elements:
            element = elements[0]
            url = element.get_attribute('href')

    except (NoSuchElementException, TimeoutException):
        pass  

    return molecule, url

def get_best_urls(cas_arr):
    '''
    Parameters
    ----------
    An array of CAS values

    Returns
    -------
    An array of associated top matches
    '''
    
    best_matches = []
    driver = setup_webdriver()  
    try:
        for cas in cas_arr:
            molecule, url = get_inxight_url(cas, driver)
            best_matches.append((molecule, url))
    finally:
        driver.quit()  

    return best_matches

In [8]:
links = get_best_urls(cas_arr)

In [9]:
links

[('853426-35-4', 'https://drugs.ncats.io/drug/4FR53SIF3A'),
 ('394730-60-0', 'https://drugs.ncats.io/drug/89BT58KELH'),
 ('10043-35-3', 'https://drugs.ncats.io/drug/R57ZHV85D4'),
 ('179324-69-7', 'https://drugs.ncats.io/drug/69G8BD63PP'),
 ('147536-97-8', 'https://drugs.ncats.io/drug/XUL93R30K2'),
 ('380843-75-4', 'https://drugs.ncats.io/drug/5018V4AEZ0'),
 ('173740-48-2', 'https://drugs.ncats.io/drug/80VUN7L00C'),
 ('914088-09-8', 'https://drugs.ncats.io/drug/7XL5ISS668'),
 ('59-41-6', 'https://drugs.ncats.io/drug/RZR75EQ2KJ'),
 ('913611-97-9', 'https://drugs.ncats.io/drug/2J3YBM1K8C'),
 ('1197953-54-0', 'https://drugs.ncats.io/drug/HYW8DB273J'),
 ('633-03-4', 'https://drugs.ncats.io/drug/G0L543D370'),
 ('59803-98-4', 'https://drugs.ncats.io/drug/E6GNX3HHTE'),
 ('138890-62-7', 'https://drugs.ncats.io/drug/9451Z89515'),
 ('357336-20-0', 'https://drugs.ncats.io/drug/U863JGG2IA'),
 ('108333-82-0', 'https://drugs.ncats.io/drug/80VUN7L00C'),
 ('35531-88-5', 'https://drugs.ncats.io/drug/5V2

In [10]:
len(links)

241

In [11]:
identifiers = [url.split('/')[-1] for _, url in links]
identifiers = [identifier if identifier != 'A' else 'MISSING' for identifier in identifiers]
identifiers

['4FR53SIF3A',
 '89BT58KELH',
 'R57ZHV85D4',
 '69G8BD63PP',
 'XUL93R30K2',
 '5018V4AEZ0',
 '80VUN7L00C',
 '7XL5ISS668',
 'RZR75EQ2KJ',
 '2J3YBM1K8C',
 'HYW8DB273J',
 'G0L543D370',
 'E6GNX3HHTE',
 '9451Z89515',
 'U863JGG2IA',
 '80VUN7L00C',
 '5V278481KE',
 'F6RJL8B278',
 '21925K482H',
 '05JZI7B19X',
 'U68WG3173Y',
 'R12CBM0EIZ',
 '8HO6PVN24W',
 '01YAE03M7J',
 '81G40H8B0T',
 '8NF31401XG',
 '0K47UL67F2',
 '3SJ3U7J6V2',
 'AY96EDV46J',
 'F0XDI6ZL63',
 'NU23Q531G1',
 'S2F83W92TK',
 '4W9K63FION',
 '5UVC90J1LK',
 'Q3JTX2Q7TU',
 'H5RIZ3MPW4',
 'O5CB12L4FN',
 'YYX637R279',
 '9LSH52S3LQ',
 'OFM06SG1KO',
 '144O8QL0L1',
 'VVJ6673MHY',
 'COF19H7WBK',
 '7QID3E7BG7',
 '4KV4X8IF6V',
 'K3GDH6OH08',
 'RRW32X4U1F',
 '46M3EV8HHE',
 '731DCA35BT',
 'FB0C1XZV4Y',
 '3ZZ5BJ9F2Q',
 'T2DHJ9645W',
 '7C546U4DEN',
 'S8A06QG2QE',
 'E90NZP2L9U',
 '73K4184T59',
 'N9I9HDB855',
 'D0Y70F2B9J',
 'R4Z9X1N2ND',
 '82WI2L7Q6E',
 '2OYE00PC25',
 'DKO1W9H7M1',
 '4AZL56CU0F',
 '5587267Z69',
 '8SSC91326P',
 'VTD58H1Z2X',
 '45TPJ4MB

In [12]:
# def extract_conditions_and_phases(data):
#     conditions_list = []
#     highest_approval_list = []

#     for entry in data:
#         if entry['name'] == 'Conditions' and 'value' in entry:
#             condition_info = entry['value']
            
#             # Extract the condition name
#             label = condition_info.get('label')
#             if label:
#                 conditions_list.append(label)
            
#             # Extract the highest phase of approval
#             highest_phase = condition_info.get('highestPhase')
#             if highest_phase:
#                 highest_approval_list.append(highest_phase)

#     return conditions_list, highest_approval_list

def extract_conditions_and_phases(data):
    """Extract conditions and their highest phases of approval from the API data."""
    conditions_list = []
    highest_approval_list = []

    for entry in data:
        if isinstance(entry, dict) and entry.get('name') == 'Conditions' and 'value' in entry:
            condition_info = entry['value']
            
            # Extract the condition name
            label = condition_info.get('label')
            if label:
                conditions_list.append(label)
            
            # Extract the highest phase of approval
            highest_phase = condition_info.get('highestPhase')
            if highest_phase:
                highest_approval_list.append(highest_phase)

    return conditions_list, highest_approval_list

In [13]:
def extract_event_details(data):
    event_details = {}
    
    for item in data:
        if 'value' in item and isinstance(item['value'], dict):  # Ensure 'value' is a dictionary
            details = item['value']
            if 'status' in details and 'sourceID' in details:
                if item['name'] == 'Highest Development Event' or item['name'] == 'Earliest Approved Event':
                    # Gather additional details
                    source_id = details.get('sourceID', 'No Source ID')
                    source_url = details.get('sourceURL', 'No Source URL')
                    
                    detail_info = {
                        'Status and Year': f"{details['status']} {details.get('year', '')}",
                        'Source ID': source_id,
                        'Source URL': source_url
                    }
                    
                    # Use the 'name' of the event as the key in the dictionary
                    event_details[item['name']] = detail_info
    
    return event_details

In [14]:
# def extract_events(identifiers):
#     events = []

#     for identifier in identifiers:
#         if identifier == "MISSING":
#             events.append({'Highest Development Event': {'Status and Year': 'Possibly Marketed Outside US 1996',
#    'Source ID': 'ANDA040166',
#    'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c0ec47b7-c13a-4fa0-91fe-d7a03c70aec9'},
#   'Earliest Approved Event': {'Status and Year': 'Possibly Marketed Outside US 1996',
#    'Source ID': 'ANDA040166',
#    'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c0ec47b7-c13a-4fa0-91fe-d7a03c70aec9'}})
#         data = get_additional_data(identifier)
#         print(extract_event_details(data))
#         events.append(extract_event_details(data))

#     return events
        
# events_arr = extract_events(identifiers)

def extract_events(identifiers):
    events = []
    acc = 0

    for identifier in identifiers:
        acc += 1
        print(acc)
        if identifier == "MISSING":
            # Create a dictionary where all details are 'MISSING'
            missing_details = {
                'Highest Development Event': {
                    'Status and Year': 'N/A',
                    'Source ID': 'N/A',
                    'Source URL': 'N/A'
                },
                'Earliest Approved Event': {
                    'Status and Year': 'N/A',
                    'Source ID': 'N/A',
                    'Source URL': 'N/A'
                }
            }
            events.append(missing_details)
        else:
            data = get_additional_data(identifier)
            event_details = extract_event_details(data)
            events.append(event_details)

    return events

In [15]:
def extract_conditions(identifiers):
    """Extract conditions for a list of identifiers."""
    conditions = []

    for identifier in identifiers:
        if identifier == "MISSING":
            conditions.append((['N/A'], ['N/A']))
        else:
            data = get_additional_data(identifier)
            if data:  # Check if data is not empty
                extracted_data = extract_conditions_and_phases(data)
                conditions.append(extracted_data)
            else:
                conditions.append(([], []))  # Append empty lists if no data
    return conditions

In [16]:
import requests

def get_additional_data(id):
    r = requests.get(f'https://drugs.ncats.io/api/v1/substances({id})/@additional')
    if 200 == r.status_code:
        return r.json()
    return '{}'

In [17]:
events_arr = extract_events(identifiers)
events_arr

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241


[{},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {'Highest Development Event': {'Status and Year': 'N/A',
   'Source ID': 'N/A',
   'Source URL': 'N/A'},
  'Earliest Approved Event': {'Status and Year': 'N/A',
   'Source ID': 'N/A',
   'Source URL': 'N/A'}},
 {'Highest Development Event': {'Status and Year': 'N/A',
   'Source ID': 'N/A',
   'Source URL': 'N/A'},
  'Earliest Approved Event': {'Status and Year': 'N/A',
   'Source ID': 'N/A',
   'Source URL': 'N/A'}},
 {'Highest Development Event': {'Status and Year': 'N/A',
   'Source ID': 'N/A',
   'Source URL': 'N/A'},
  'Earliest Approved Event': {'Status and Year': 'N/A',
   'Source ID': 'N/A'

In [18]:
import requests

def get_additional_data(id):
    r = requests.get(f'https://drugs.ncats.io/api/v1/substances({id})/@additional')
    if 200 == r.status_code:
        return r.json()
    return ([], [])

In [19]:
conditions_arr = extract_conditions(identifiers)
conditions_arr

[([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 ([], []),
 (['N/A'], ['N/A']),
 (['N/A'], ['N/A']),
 (['N/A'], ['N/A']),
 (['N/A'], ['N/A']),
 (['N/A'], ['N/A']),
 (['N/A'], ['N/A']),
 (['N

In [20]:
data = []
for event in events_arr:
    record = {}
    for key, value in event.items():
        for sub_key, sub_value in value.items():
            record[f"{key} {sub_key}"] = sub_value
    data.append(record)

df = pd.DataFrame(data)

In [21]:
rows = [{'conditions': conditions, 'phases': phases} for conditions, phases in conditions_arr]

# Create DataFrame
conditions_df = pd.DataFrame(rows)
conditions_df['conditions'] = conditions_df['conditions'].apply(lambda x: '; '.join(x))
conditions_df['phases'] = conditions_df['phases'].apply(lambda x: '; '.join(x))
conditions_df

,conditions,phases
0,,
1,,
2,,
3,,
4,,
...,...,...
236,N/A,N/A
237,N/A,N/A
238,N/A,N/A
239,N/A,N/A


In [22]:
merged = pd.concat([df, conditions_df], axis=1)
merged

,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,NaN,NaN,NaN,NaN,NaN,NaN,,
1,NaN,NaN,NaN,NaN,NaN,NaN,,
2,NaN,NaN,NaN,NaN,NaN,NaN,,
3,NaN,NaN,NaN,NaN,NaN,NaN,,
4,NaN,NaN,NaN,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...
236,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
237,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
238,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
239,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [23]:
links_df = pd.DataFrame(links, columns=['CAS', 'URL'])
links_df

,CAS,URL
0,853426-35-4,https://drugs.ncats.io/drug/4FR53SIF3A
1,394730-60-0,https://drugs.ncats.io/drug/89BT58KELH
2,10043-35-3,https://drugs.ncats.io/drug/R57ZHV85D4
3,179324-69-7,https://drugs.ncats.io/drug/69G8BD63PP
4,147536-97-8,https://drugs.ncats.io/drug/XUL93R30K2
...,...,...
236,56211-40-6,N/A
237,89778-26-7,N/A
238,192391-48-3,N/A
239,114899-77-3,N/A


In [24]:
result = pd.concat([links_df, merged], axis=1)
result

,CAS,URL,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,853426-35-4,https://drugs.ncats.io/drug/4FR53SIF3A,NaN,NaN,NaN,NaN,NaN,NaN,,
1,394730-60-0,https://drugs.ncats.io/drug/89BT58KELH,NaN,NaN,NaN,NaN,NaN,NaN,,
2,10043-35-3,https://drugs.ncats.io/drug/R57ZHV85D4,NaN,NaN,NaN,NaN,NaN,NaN,,
3,179324-69-7,https://drugs.ncats.io/drug/69G8BD63PP,NaN,NaN,NaN,NaN,NaN,NaN,,
4,147536-97-8,https://drugs.ncats.io/drug/XUL93R30K2,NaN,NaN,NaN,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...
236,56211-40-6,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
237,89778-26-7,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
238,192391-48-3,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
239,114899-77-3,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [25]:
result.tail(10)

,CAS,URL,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
231,150683-30-0,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
232,56-40-6,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
233,137-58-6,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
234,97240-79-4,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
235,123948-87-8,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
236,56211-40-6,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
237,89778-26-7,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
238,192391-48-3,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
239,114899-77-3,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
240,123154-38-1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [26]:
result.to_csv("inxight_data_3.csv")